In [ ]:
from IPython.core.display import HTML
with open('style.html', 'r') as file:
     css = file.read()
HTML(css)

# Spieler gegen KI

## Für das Notebook benötigte Imports
* `ipywidgets`: Anzeige von Buttons und ähnlichen Widgets.
* `display`, `clear_output`: Funktionen aus der IPython.display Library die zur besseren Darstellung verwendet werden.
* `chess`: Die Schachbibliothek, welche die Funktion des Schachspiels verwaltet.
* `time`: Wird für Verzögerungen, welche der besseren Nutzbarkeit dienen verwendet.
* `datetime`: Wird für die Benennung der Play-History verwendet.
* `json`: Wird für den Import der Endspiel-Datenbank verwendet.

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import chess
import time
from datetime import datetime
from chess import Termination
import json

## Erstellung eines eigenen Spielfeldes

Zur Erstellung eines individuellen Spielfeldes werden zwei Dictionaries verwendet. Diese
besitzen jeweils als `key` die Figur, die auf dem Spielfeld platziert werden soll. Als `value` besitzen die Einträge
jeweils eine Liste, die mit Namen der Felder gefüllt werden müssen, auf denen die jeweilige Figur stehen soll.

In [ ]:
WHITE_POSITIONS = {'king':['e1'],
                   'queen':[],
                   'rooks':['a1'],
                   'bishops':[],
                   'knights':[],
                   'pawns':[]}

BLACK_POSITIONS = {'king':['e8'],
                   'queen':[],
                   'rooks':[],
                   'bishops':[],
                   'knights':[],
                   'pawns':[]}

## Die *get_occupied_cells* Hilfsfunktion
Die Funktion übersetzt die zuvor erstellten Dictionaries in eine Liste, die alle `values` der
Dictionaries enthält.

Funktionsparameter:
* Keine

Nebeneffekte:
* Keine

Ergebnis der Ausführung:
* Eine Liste aller verwendeten Zellen

In [ ]:
def get_occupied_cells():
    cells = []
    for values in WHITE_POSITIONS.values():
        for value in values:
            cells.append(value)
    for values in BLACK_POSITIONS.values():
        for value in values:
            cells.append(value)
    return cells

## Die *check_correct_cells* Hilfsfunktion
Die Funktion `check_for_correct_cells()` überprüft die eingegebenen Felder in den Dictionaries `WHITE_POSITIONS` und
`BLACK_POSITIONS` auf deren Korrektheit. Diese wird in Form eines booleschen Werts angegeben.

Funktionsargumente:
* Keine

Nebeneffekte:
* Keine

Ergebnis der Ausführung:
* Ein Boolscher-Wert, welcher angibt, ob alle vom Nutzer eingegebenen Werte valide sind.

In [ ]:
import re
def check_for_correct_cells():
    cells = get_occupied_cells()
    for cell in cells:
        x = re.search("[a-h][1-8]", cell)
        if x is None:
            print(cell)
            print("Value incorrect!")
            return False
        else:
            pass
    return True

## Die *create_board* Hilfsfunktion
Die Funktion `create_board()` füllt das Schachbrett mit den Figuren, die in den Dictionaries angegeben wurden. Als
Rückgabewert gibt die Funktion das gefüllte Schachbrett zurück.

Funktionsargumente:
* Keine

Nebeneffekte:
* Keine

Ergebnis der Ausführung:
* Ein Spielfeld als Objekt der `chess` Library, welches die vom Nutzer eingegebene Spielsituation darstellt.

Algorithmus:
* Wenn korrekte Zellen eingegeben wurden.
* Alle eingegebenen Positionen überprüfen und den Figurtyp wählen.
* Figur an angegebener Stelle platzieren.
* Castling Rights zurücksetzen, da Boards mit einem Turm sonst als invalide bezeichnet werden.
* Wenn das Spielbrett gegen Schach-Regeln verstößt (invalide ist) -> ersetzen durch das standard Layout
* Zurückgeben des Spielbretts.

In [ ]:
def create_board():
    local_board = chess.Board()
    occupied_cells = {}
    if check_for_correct_cells():
        for key, values in WHITE_POSITIONS.items():
            if key == 'king':
                piece_type = chess.KING
            elif key == 'queen':
                piece_type = chess.QUEEN
            elif key == 'rooks':
                piece_type = chess.ROOK
            elif key == 'bishops':
                piece_type = chess.BISHOP
            elif key == 'knights':
                piece_type = chess.KNIGHT
            else:
                piece_type = chess.PAWN
            piece = chess.Piece(piece_type, chess.WHITE)
            for value in values:
                square = chess.parse_square(value)
                occupied_cells[square] = piece
        for key, values in BLACK_POSITIONS.items():
            if key == 'king':
                piece_type = chess.KING
            elif key == 'queen':
                piece_type = chess.QUEEN
            elif key == 'rooks':
                piece_type = chess.ROOK
            elif key == 'bishops':
                piece_type = chess.BISHOP
            elif key == 'knights':
                piece_type = chess.KNIGHT
            else:
                piece_type = chess.PAWN
            piece = chess.Piece(piece_type, chess.BLACK)
            for value in values:
                square = chess.parse_square(value)
                occupied_cells[square] = piece

        local_board.set_piece_map(occupied_cells)
        local_board.castling_rights = local_board.clean_castling_rights()
        
        # Herausfinden, weshalb momentan angegebenes Board invalid ist
        if not local_board.is_valid():
            display(local_board)
            print("Specified lineup is invalid")
            print("Instead standard board created")
            local_board = chess.Board()
    return local_board

## Die *load_s_n_sequence* Hilfsfunktion zum Import der Daten
Die Daten wurden als FEN in der JSON-Datei Serialisiert.
Zum Initialisieren der Liste werden alle FENs aus einer Datei gelesen und Board-Objekte erstellt.

Funktionsparameter:
* `filepath`: Pfad zur Datei, welche die $S_n$ Sequenz enthält

Nebeneffekte:
* Keine

Ergebnis der Ausführung:
* Listen Objekt mit der $S_n$ Sequenz

In [ ]:
def load_s_n_sequence(filepath):
    s_n_sequence_new = []
    f = open(filepath, "r")
    tmp = json.loads(f.read())
    for item in tmp:
        tmp_list = []
        for board in item:
            tmp_list.append(chess.Board(board))
        s_n_sequence_new.append(tmp_list)
    f.close()
    return s_n_sequence_new

## die *find_board_in_sequence* Hilfsfunktion

Diese Funktion durchsucht eine $S_n$ Sequenz nach dem ersten Vorkommen eines übergebenen Board-Objekts.

Funktionsargumente:
* situation: Das Board (als Objekt), welches gefunden werden soll
* sequence: Die $S_n$ Sequenz, in welcher das Board gesucht wird

Ergebnis der Ausführung:
* Die Funktion hat zwei mögliche Rückgaben:
  * Ein Tupel mit $S_n$ Index (z.B. $S_3$) und Board-Index (z.B. 100).
    Dieses Tupel drückt aus, wo in der Sequenz das Board gefunden wurde.
  * Das Tupel (-1,-1). Dies drückt aus, dass das Board nicht gefunden wurde.

Nebeneffekte:
Die Funktion verändert keinen der übergebenen Parameter.

Algorithmus:
1. Über die $S_n$ Sequenz iterieren.
2. Über jedes Board in einem spezifischem $S_n$ iterieren.
3. Das Board mit dem `situation` Objekt vergleichen.
    1. Wenn das Board übereinstimmt, die Indizes zurückgeben.
    2. Wenn das Board nicht übereinstimmt, weitersuchen.

In [ ]:
def find_board_in_sequence(situation, sequence):
    board_str = (situation.turn, situation.__str__())

    for i in range(len(sequence)):
        for j in range(len(sequence[i])):
            item = sequence[i][j]
            if board_str == (item.turn, item.__str__()):
                return i, j
    return -1,-1

## Die *find_next_move* Hilfsfunktion
Diese Funktion berechnet aus einem übergebenen Board den idealen Spielzug.
Aus Effizienzgründen gibt sie auch die Position des nächsten Boards in der $S_n$ Sequenz zurück.

Funktionsparameter:
* `curr_board`: Das Board, für welches der nächste Spielzug berechnet werden soll.
* `s_index`: Das $n$ eines $S_n$, in welchem sich `curr_board` befindet.
Wird aus effizienz Gründen übergeben.
* `s_n_sequence`: Die Liste mit allen $S_n$

Ergebnis der Ausführung:
* Rückgabe: Der nächsten Move oder der Wert -1 als Hinweis, dass kein Spielzug berechnet werden konnte

Nebeneffekte:
* Keine.

Algorithmus:
* Führe alle möglichen moves durch, bis das Ergebnis-Board in $S_{n-1}$ gefunden wurde.
* Gebe den neuen s_index, board_index und den gefundenen move zurück.


In [ ]:
def find_next_move(curr_board, s_index, s_n_sequence):
    for move in curr_board.legal_moves:
        curr_board.push(move)
        _tmp, board_index_tmp = find_board_in_sequence(curr_board, [s_n_sequence[s_index - 1]])
        s_index_tmp = s_index - 1
        if board_index_tmp != -1:
            curr_board.pop()
            return s_index_tmp, board_index_tmp, move
        curr_board.pop()

    return -1, -1, None

## Globale Variablen
Hierbei handelt es sich um Werte, welche für die Funktion der KI benötigt werden.

In [ ]:
FILE = "S_n_seq_12_01.json"
S_N_SEQUENCE = load_s_n_sequence(FILE)

S_INDEX = 0
BOARD_INDEX = 0

board = create_board()
filename = ''

## Setup der Steuerung
Ergänzend zu dem bereitgestellten UI durch die `python-chess` Bibliothek werden noch weitere UI-Elemente benötigt, die
in Form von globalen Variablen definiert worden sind:
- `input_field` = Ein Eingabefeld, in dem der nächste zug von weiß eingetragen werden soll.
- `execute_button` = Ein Button, der nach der Auswahl des Zuges diesen auch auf dem Schachbrett ausführt.

In [ ]:
input_field = widgets.Text()

execute_button = widgets.Button(
    description='Execute Move',
    disabled=False,
    button_style='',
    tooltip='Executes the move selected with the dropdowns "Piece:" and "Move:"',
    icon='check'
)

## Die *get_color* Hilfsfunktion
Innerhalb der python-chess library werden die Farben des Schachbretts `chess.WHITE` und `chess.BLACK` als boolesche
Variablen definiert. Deswegen wurde zur späteren Dokumentation, aber auch zum Debuggen eine Funktion geschrieben,
die die Farbe, die gerade am Zug ist, als String zurückgibt.

Funktionsparameter:
* `turn`: Boolscher wert, welcher den Spieler am Zug in der `chess` Library repräsentiert.

Ergebnis der Ausführung:
* Ein String welcher Spieler am Zug ist.

In [ ]:
def get_color(turn):
    if turn:
        return "White"
    else:
        return "Black"

## Die *update_filename* Hilfsfunktion
Zum Beschreiben der globalen Variablen `filename` wurde die Funktion `update_filename` definiert.

Funktionsparameter:
* `new_value`: Der neue Wert

Nebeneffekte:
* Der Wert der globalen Variable `filename` wird verändert.

Ergebnis der Ausführung:
* Der Wert von `new_value` befindet sich in der Variable `filename`

In [ ]:
def update_filename(new_value):
    global filename
    filename = new_value

## Die *reset_input_field* Hilfsfunktion
Die Funktion setzt das UI Element auf ihren Standardwert zurück.

Funktionsargumente:
* Keine

Nebeneffekte:
* Der Wert in der globalen Variable `input_field` wird gelöscht.

Ergebnis der Ausführung:
* Der Wert in `input_field` wird entfernt.

In [ ]:
def reset_input_field():
    global input_field
    input_field.value = ''

## Die *reset_board* Hilfsfunktion
Die Funktion setzt das Schachbrett auf den Zustand zu Beginn der vom Nutzer eingegebenen Situation zurück.

Funktionsargumente:
* Keine

Nebeneffekte:
* Der Zustand in der globalen Variable `board` geht verloren.
* Die Inhalte von `S_INDEX` und `BOARD_INDEX` werden überschrieben.
* Ggf. wird ein Zug am Board durchgeführt.

Ergebnis der Ausführung:
* Das board Objekt wird neu erstellt
* Wenn Weiß am Zug ist, wird der erste Zug von der KI durchgeführt.


In [ ]:
def reset_board():
    global board
    board = create_board()
    global S_INDEX
    global BOARD_INDEX
    S_INDEX, BOARD_INDEX = find_board_in_sequence(board, S_N_SEQUENCE)
    if board.turn:
        S_INDEX, BOARD_INDEX, move = find_next_move(board, S_INDEX, S_N_SEQUENCE)
        execute_move(move, board.turn)
        reload_screen()


## Die *execute_move* Hilfsfunktion
Die Funktion `execute_move` erhält als Parameter einen `chess.Move` und führt diesen auf dem globalen `board` für die
Farbe `chess.turn` aus. Zusätzlich wird in der Datei, die in der globalen Variablen `filename` zu finden ist, für
den Zug ein Eintrag hinterlegt.

Funktionsargumente:
* `move`: Der durchzuführende move.
* `turn`: Der Spieler, der am Zug ist

Nebeneffekte:
* Die Inhalte von `board`, `S_INDEX` und `BOARD_INDEX` werden verändert.
* Die Datei `Played_Games/filename` wird verändert

Ergebnis der Ausführung:
* Ein Zug wird gespielt
* Die Ausführung wird gespeichert

In [ ]:
def execute_move(move, turn):
    global board
    global S_INDEX
    global BOARD_INDEX
    move_file = open("Played_Games/" + filename, "a")
    if turn:
        move_file.write(str(board.fullmove_number) + ". " + move.uci() + " ")
    else:
        move_file.write(move.uci() + "\n")
    move_file.close()
    board.push(move)
    S_INDEX, BOARD_INDEX = find_board_in_sequence(board, S_N_SEQUENCE)

## Die *get_pieces_placed_on_board* Hilfsfunktion

Das Ziel der Funktion `get_pieces_placed_on_board` besteht darin von einer mitgegebenen `chess.Color` die Figuren zu
bestimmen, die noch auf dem Schachbrett stehen. Zurück gibt sie ein Dictionary, das als `key` die Schachfiguren und
als `value` eine Liste mit den Positionen der Figuren besitzt.

Funktionsargumente:
* `color`: Die Farbe des Spielers, dessen Figuren gefunden werden sollen

Nebeneffekte:
* Keine

Ergebnis der Ausführung:
* Ein Liste mit Figuren und Positionen dieser Figuren

In [ ]:
def get_pieces_placed_on_board(color):
    piece_type_to_string = {
        1 : "Pawn",
        2 : "Knight",
        3 : "Bishop",
        4 : "Rook",
        5 : "Queen",
        6 : "King"
    }
    pieces_with_position = {
        "Pawn" : [],
        "Knight" : [],
        "Bishop" : [],
        "Rook" : [],
        "Queen" : [],
        "King" : []
    }
    for piece_square ,color_piece in board.piece_map().items():
        if color_piece.color == color:
            pieces_with_position[piece_type_to_string[color_piece.piece_type]].append(chess.square_name(piece_square))
    return pieces_with_position

## Die *get_moves_from Square* Hilfsfunktion
Bei der Funktion `get_moves_from_square` werden anhand der mitgegebenen `legal_moves` für ein vorgegebenes Feld die
Züge selektiert, die man von diesem Feld aus ziehen kann.

Funktionsargumente:
* `square`: Das Spielfeld, von welchem mögliche Züge bestimmt werden sollen.
* `legal_moves`: Die Spielzüge die für das aktuelle Spielbrett erlaubt sind.

Nebeneffekte:
* Keine

Ergebnis der Ausführung: 
* Eine Liste aller möglichen Züge ausgehend von einem Feld.

In [ ]:
def get_moves_from_square(square, legal_moves):
    moves = []
    for possible_move in legal_moves:
        if square == chess.square_name(possible_move.from_square):
            moves.append(possible_move)
    return moves

## Die *reload_screen* Hilfsfunktion
Da die Anzeige des UI auf Konsolenausgaben basiert, muss diese auch nach einem neuen Zug geleert werden, sodass die
Konsole nicht mit mehreren Elementen überflutet wird. Hierfür aktualisiert die Funktion `reload_screen` die Ausgabe und
zeigt das Schachbrett, das `input_field` und den Knopf zum Ausführen des Zugs erneut an.

Funktionsargumente:
* Keine

Nebeneffekte:
* Der Inhalt des Bildschirms wird aktualisiert

* Ergebnis der Ausführung:
* Inhalte werden entfernt und mit aktuellen Werten dargestellt

In [ ]:
def reload_screen():
    clear_output()
    display(board, input_field, execute_button)

## Die *show_end_screen* Hilfsfunktion
Die Funktion `show_end_screen` das Schachbrett an.

Funktionsargumente:
* Keine

Nebeneffekte:
* `input_field` und `execute_button` werden nicht mehr angezeigt

In [ ]:
def show_end_screen():
    clear_output()
    display(board)

## Die *execute_entered_move* Hilfsfunktion
Für den `execute_button` und das `input_field` wurde eine Funktion geschrieben, die für das Ausführen des Zuges
verantwortlich ist (Entweder durch Klicken des Buttons oder Drücken der Enter-Taste). Weiterhin wird auch in dieser
Funktion überprüft, ob das Spiel bereits beendet wurde.

Funktionsparameter:
* `change`: Die change Variable beinhaltet die Parameter des auslösenden Events

Nebeneffekte:
* Die Werte in den globalen Variablen `board`, `S_INDEX` und `BOARD_INDEX` werden verändert.
* Die Anzeige wird verändert.
* Die Datei `Played_Games/filename` wird verändert.

Ergebnis der Ausführung:
* Die Eingabe des Nutzers wird umgesetzt.
* Es wird ein Zug für die KI berechnet und umgesetzt.
* Die Züge werden gespeichert.
* Die Ansicht für den Nutzer wird aktualisiert.

In [ ]:
def execute_entered_move(change):
    global board
    global S_INDEX
    global BOARD_INDEX
    try:
        if input_field.value != '':
            black_move = chess.Move.from_uci(input_field.value)
            if black_move in list(board.legal_moves):
                execute_move(black_move, board.turn)
                reset_input_field()
                # Next AI move executed by white
                S_INDEX, BOARD_INDEX, next_move = find_next_move(board, S_INDEX, S_N_SEQUENCE)
                if S_INDEX == -1:
                    print("No Move for white found")
                else:
                    execute_move(next_move, board.turn)
                    reload_screen()
                if board.legal_moves.count() == 0:
                    # If wanted add different endings
                    show_end_screen()
                    result = board.outcome().termination
                    if result == Termination.CHECKMATE:
                        print(get_color(board.turn) + " has lost because of Checkmate!")
                    elif result == Termination.STALEMATE:
                        print("It's a draw!")
                    elif result == Termination.INSUFFICIENT_MATERIAL:
                        print("No side can win the game anymore!")
                    elif result == Termination.SEVENTYFIVE_MOVES:
                        print("The game is drawn because half-move clock is greater than 150 since a capture or a pwn has been moved.")
                    elif result == Termination.FIVEFOLD_REPETITION:
                        print("The game is drawn because the current position occurred the fifth time!")
                    elif result == Termination.FIFTY_MOVES:
                        print("The game is drawn because half-move clock is greater than 100 since a capture or a pwn has been moved.")
                    elif result == Termination.THREEFOLD_REPETITION:
                        print("The game is drawn because the current position occurred the third time!")
                    elif result == Termination.VARIANT_WIN:
                        print(get_color(board.turn) + " has won because of variant-specific conditions")
                    elif result == Termination.VARIANT_LOSS:
                        print(get_color(board.turn) + " has lost because of variant-specific conditions")
                    elif result == Termination.VARIANT_DRAW:
                        print("Game is drawn because of variant-specific conditions!")
                    else:
                        print("Something went wrong!")
            else:
                print("Entered a wrong move. Please try again!")
                print(input_field.value)
                time.sleep(2)
                reload_screen()
        else:
            print("Enter a move!")
            time.sleep(2)
            reload_screen()
    except ValueError:
        print("Entered a wrong move. Please try again!")
        print(input_field.value)
        time.sleep(2)
        reload_screen()

## Zuweisen der Funktion zu den eingabemöglichkeiten
Sodass die Funktionen letztendlich auch an die UI-Elemente gebunden sind, werden im folgenden Codeabschnitt die Funktionen den Objekten zugewiesen.

In [ ]:
execute_button.on_click(execute_entered_move)
input_field.on_submit(execute_entered_move)

## Die *start_game* Funktion
Die Funktion `start_game` bereitet jegliche Parameter für einen Spielverlauf vor. Dazu gehört:
- Erstellung eines neuen Boards.
- Das Erstellen einer neuen Historie für das neu begonnene Spiel

Funktionsargumente:
* Keine

Nebeneffekte:
* Das Spiel wird gestartet, die Nebeneffekte der Funktionen in diesem Notebook fallen an.

Ergebnis der Ausführung:
* Der Zustand des Spiels wird zurückgesetzt und das Spiel gestartet.

In [ ]:
def start_game():
    update_filename("Move-History_" + str(datetime.today().replace(microsecond=0)).replace(":","_") + ".txt")
    move_file = open("Played_Games/" + filename, "a")
    reset_board()
    move_file.write(board.fen() + "\n")
    reload_screen()

Mit dem Aufruf der Funktion `start_game` kann nun ein Spiel gegen die KI gestartet werden.

In [ ]:
start_game()